In [ ]:
import math
import copy

class AllRoutes:
    def __init__(self,n):
        self._dist = [[math.inf]*n for _ in range(n)]
        for i in range(n):
            self._dist[i][i] = 0
        self._n = n

    def add_road(self,a,b,x):
        if x < self._dist[a-1][b-1]:
            self._dist[a-1][b-1] = x
            self._dist[b-1][a-1] = x

    def get_table(self):
        for k in range(self._n):
            for i in range(self._n):
                for j in range(self._n):
                    newdist = min(self._dist[i][j],
                                            self._dist[i][k] + self._dist[k][j])
                    self._dist[i][j] = newdist
                    self._dist[j][i] = newdist
        res = copy.deepcopy(self._dist)
        for y in range(len(res)):
            for x in range(len(res[y])):
                if math.isinf(res[y][x]):
                    res[y][x] = -1
        return res

if __name__ == "__main__":
    a = AllRoutes(4)
    a.add_road(1,2,2)
    a.add_road(1,3,5)
    a.add_road(2,3,1)
    print(a.get_table())
    # [[0,2,3,-1],[2,0,1,-1],[3,1,0,-1],[-1,-1,-1,0]]

    b = AllRoutes(5)
    b.add_road(3,4,6)
    b.add_road(4,5,5)
    b.add_road(4,5,6)
    b.add_road(1,5,7)
    b.add_road(1,4,7)
    b.add_road(4,5,1)
    b.add_road(3,4,8)
    b.add_road(2,3,6)
    b.add_road(4,5,4)
    print(b.get_table())
    # [[0, 19, 13, 7, 7], [19, 0, 6, 12, 13], [13, 6, 0, 6, 7], [7, 12, 6, 0, 1], [7, 13, 7, 1, 0]]

In [ ]:
# bellman_ford

import math
from random import randint, randrange
from time import perf_counter

n = 100000
m = 10*n
kaaret = []
etaisyys = [math.inf]*n
etaisyys[0] = 0
kierrokset = 0

for i in range(m):
    alku = randrange(n)
    loppu = randrange(n)
    paino = randint(1, 100)
    kaaret.append((alku, loppu, paino))

aloitus = perf_counter()
while True:
    muutos = False
    for kaari in kaaret:
        nyky = etaisyys[kaari[1]]
        uusi = etaisyys[kaari[0]] + kaari[2]
        if uusi < nyky:
            etaisyys[kaari[1]] = uusi
            muutos = True
    kierrokset += 1
    if not muutos:
        break
lopetus = perf_counter()

print("kierroksia:", kierrokset)
print("aika:", (lopetus - aloitus))

In [ ]:
# bestroute
import math
from heapq import heappush, heappop

class BestRoute:
    def __init__(self,n):
        self._verkko = [[] for _ in range(n + 1)]
        self._n = n + 1

    def add_road(self,a,b,x):
        self._verkko[a].append((b, x))
        self._verkko[b].append((a, x))

    def find_route(self,a,b):
        keko = []
        kasitelty = [False]*self._n
        etaisyys = [math.inf]*self._n
        etaisyys[a] = 0
        heappush(keko, (0, a))
        while len(keko) != 0:
            solmu = heappop(keko)[1]
            if kasitelty[solmu]:
                continue
            kasitelty[solmu] = True
            for kaari in self._verkko[solmu]:
                nyky = etaisyys[kaari[0]]
                uusi = etaisyys[solmu] + kaari[1]
                if uusi < nyky:
                    etaisyys[kaari[0]] = uusi
                    heappush(keko, (uusi, kaari[0]))
        return etaisyys[b] if math.isfinite(etaisyys[b]) else -1

if __name__ == "__main__":
    b = BestRoute(3)
    b.add_road(1,2,2)
    print(b.find_route(1,3)) # -1
    b.add_road(1,3,5)
    print(b.find_route(1,3)) # 5
    b.add_road(2,3,1)
    print(b.find_route(1,3)) # 3

In [ ]:
# breakwall
import math
from heapq import heappush, heappop


class Labyrinth:
    def __init__(self, r):
        self._r = r
        self._m = len(r)
        self._n = len(r[0])
        for y in range(self._m):
            ax = r[y].find("A")
            if ax != -1:
                self._aloc = (ax, y)
            bx = r[y].find("B")
            if bx != -1:
                self._bloc = (bx, y)
        self._neighbours = {}
        for y in range(1, self._m - 1):
            for x in range(1, self._n - 1):
                self._neighbours[(x, y)] = []
                self._check_neighbour((x, y), (x - 1, y))
                self._check_neighbour((x, y), (x + 1, y))
                self._check_neighbour((x, y), (x, y - 1))
                self._check_neighbour((x, y), (x, y + 1))

    def _check_neighbour(self, here, neighbour):
        square = self._r[neighbour[1]][neighbour[0]]
        if square == "*":
            self._neighbours[here].append((neighbour, 1))
        elif square == "." or square == "A" or square == "B":
            self._neighbours[here].append((neighbour, 0))

    def route(self):
        heap = []
        visited = dict.fromkeys(self._neighbours, False)
        distances = dict.fromkeys(self._neighbours, math.inf)
        distances[self._aloc] = 0
        heappush(heap, (0, self._aloc))
        while len(heap) != 0:
            vtx = heappop(heap)[1]
            if visited[vtx]:
                continue
            visited[vtx] = True
            for edge in self._neighbours[vtx]:
                curr = distances[edge[0]]
                new = distances[vtx] + edge[1]
                if new < curr:
                    distances[edge[0]] = new
                    heappush(heap, (new, edge[0]))
        return distances[self._bloc]


def count(r):
    lab = Labyrinth(r)
    return lab.route()


if __name__ == "__main__":
    r = [
        "########",
        "#*A*...#",
        "#.*****#",
        "#.**.**#",
        "#.*****#",
        "#..*.B.#",
        "########",
    ]
    print(count(r))  # 2

In [ ]:
# floyd_warshall
import math
from random import randint, randrange
from time import perf_counter

n = 1000
m = 10*n
kaaret = []
etaisyys = [[math.inf]*n for _ in range(n)]
for i in range(n):
    etaisyys[i][i] = 0
kierrokset = 0

for i in range(m):
    alku = randrange(n)
    while True:
        loppu = randrange(n)
        if loppu != alku:
            break
    paino = randint(1, 100)
    kaaret.append((alku, loppu, paino))
    etaisyys[alku][loppu] = paino

aloitus = perf_counter()
for k in range(n):
    for i in range(n):
        for j in range(n):
            etaisyys[i][j] = min(etaisyys[i][j], etaisyys[i][k]+etaisyys[k][j])
lopetus = perf_counter()

print("aikaa kului:", (lopetus - aloitus))

In [ ]:
# listjump
import math
from heapq import heappush, heappop

class BestRoute:
    def __init__(self,n):
        self._verkko = [[] for _ in range(n)]
        self._n = n

    def add_jump(self,a,b,x):
        self._verkko[a].append((b, x))

    def find_route(self,a,b):
        keko = []
        kasitelty = [False]*self._n
        etaisyys = [math.inf]*self._n
        etaisyys[a] = 0
        heappush(keko, (0, a))
        while len(keko) != 0:
            solmu = heappop(keko)[1]
            if kasitelty[solmu]:
                continue
            kasitelty[solmu] = True
            for kaari in self._verkko[solmu]:
                nyky = etaisyys[kaari[0]]
                uusi = etaisyys[solmu] + kaari[1]
                if uusi < nyky:
                    etaisyys[kaari[0]] = uusi
                    heappush(keko, (uusi, kaari[0]))
        return etaisyys[b] if math.isfinite(etaisyys[b]) else -1

def calculate(t):
    r = BestRoute(len(t))
    for i in range(len(t)):
        l = t[i]
        if i - l >= 0:
            r.add_jump(i, i - l, l)
        if i + l < len(t):
            r.add_jump(i, i + l, l)
    return r.find_route(0, len(t) - 1)

if __name__ == "__main__":
    print(calculate([1,1,1,1])) # 3
    print(calculate([3,2,1])) # -1
    print(calculate([3,5,2,2,2,3,5])) # 10
    print(calculate([7,5,3,1,4,2,4,6,1])) # 32

In [ ]:
# shorthening
import math

class Shortening:
    def __init__(self,n):
        self._edges = []
        self._n = n

    def add_edge(self,a,b,x):
        self._edges.append((a-1, b-1, x))

    def check(self,a,b):
        # ajetaan Bellman-Fordia, jos tulos paranee
        #  kierroksen n-1 jälkeen niin polulla on negatiivinen sykli
        distance = [math.inf]*self._n
        distance[a-1] = 0
        i = 0
        while True:
            i += 1
            changed = False
            b_changed = False
            for edge in self._edges:
                current = distance[edge[1]]
                new = distance[edge[0]] + edge[2]
                if new < current:
                    distance[edge[1]] = new
                    if edge[1] == b-1:
                        b_changed = True
                    changed = True
            # print(distance)
            if not changed: # ei negatiivisia syklejä
                return False
            if i >= self._n and b_changed: # negatiivinen sykli b:hen johtavalla polulla
                return True
            if i > 2*self._n: # negatiivinen sykli jossain muualla kaaviossa
                return False



if __name__ == "__main__":
    s = Shortening(5)
    print(s.check(1,3)) # False
    s.add_edge(1,2,5)
    s.add_edge(2,3,-2)
    print(s.check(1,3)) # False
    s.add_edge(2,4,1)
    s.add_edge(4,2,-2)
    print(s.check(1,3)) # True

In [ ]:
# water
import math
from heapq import heappush, heappop


def count(a, b, x):
    # https://en.wikipedia.org/wiki/Water_pouring_puzzle
    # https://www.youtube.com/watch?v=0Oef3MHYEC0
    # https://webpages.uncc.edu/~hbreiter/Exotic/Decanting.pdf

    # kaikki mitattavissa olevat määrät ovat tämän monikertoja
    pienin = math.gcd(a, b)

    # onko ratkaisu yleensä mahdollinen?
    if x % pienin != 0:
        return -1

    # luodaan mahdolliset tilat
    tilat = [
        (i, j)
        for i in range(0, a + 1, pienin)
        for j in range(0, b + 1, pienin)
        if i == 0 or i == a or j == 0 or j == b
    ]

    # sallitut siirtymät
    verkko = {t: [] for t in tilat}
    for alku in tilat:
        for loppu in tilat:
            if alku == loppu:
                continue
            if alku[0] == loppu[0] and (loppu[1] == 0 or loppu[1] == b):
                # 1. astia pysyy samana, 2. täyttyy tai tyhjentyy
                verkko[alku].append((loppu, abs(loppu[1] - alku[1])))
            elif alku[1] == loppu[1] and (loppu[0] == 0 or loppu[0] == a):
                # 2. astia pysyy samana, 1. täyttyy tai tyhjentyy
                verkko[alku].append((loppu, abs(loppu[0] - alku[0])))
            elif alku[0] + alku[1] == loppu[0] + loppu[1]:
                # siirto astiasta toiseen, kokonaismäärä säilyy
                verkko[alku].append((loppu, abs(loppu[0] - alku[0])))

    # Dijkstra
    keko = []
    kasitelty = {t: False for t in tilat}
    etaisyys = {t: math.inf for t in tilat}
    etaisyys[(0, 0)] = 0  # lähdetään tilanteesta jossa molemmat astiat tyhjiä
    heappush(keko, (0, (0, 0)))
    while len(keko) != 0:
        solmu = heappop(keko)[1]
        if kasitelty[solmu]:
            continue
        kasitelty[solmu] = True
        for kaari in verkko[solmu]:
            nyky = etaisyys[kaari[0]]
            uusi = etaisyys[solmu] + kaari[1]
            if uusi < nyky:
                etaisyys[kaari[0]] = uusi
                heappush(keko, (uusi, kaari[0]))

    return min(etaisyys[(x, 0)], etaisyys[(x, b)])


if __name__ == "__main__":
    print(count(5, 4, 2))  # 22
    print(count(4, 3, 2))  # 16
    print(count(3, 3, 1))  # -1
    print(count(10, 9, 8))  # 46
    print(count(123, 456, 42))  # 10530